<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, coaddition and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mFitplane

A large percentage of the Montage modules focus on processing a set of images (often retrieved from a mission archive); projecting them to a common frame, adjusting their background levels as a set, and coadding them into a mosaic.

This page is focused on the details of one of these modules: mFitplane.  For a broader context, please see <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands.

<b>Note:</b> The MontagePy python package has no external dependencies. We include other utilities on this page to aid in visualizing MontagePy package results.


In [1]:
from MontagePy.main import mFitplane, mBackground, mViewer

help(mFitplane)

Help on built-in function mFitplane in module MontagePy.main:

mFitplane(...)
    mFitplane uses least squares to fit a plane (excluding outlier pixels) to an image.
    
    Parameters
    ----------
    input_file : str
        FITS file for plane fitting.
    levelOnly : bool, optional
        Only fit for level difference not a full plane with slopes.
    border : int, optional
        Exclude a pixel border from the fitting.
    debug : int, optional
        Debugging output level.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for errors).
    a : float
        Plane fit coefficient for X axis.
    b : float
        Plane fit coefficient for Y axis.
    c : float
        Plane fit constant offset.
    crpix1 : float
        X-axis pixel coordinate for center of region.
    crpix2 : float
        Y-axis pixel coordinate for center of region.
    xmin : float
        Minimum X-axis value.
    xmax : f

## mFitplane Example:

mFitplane least-squares fits a plane to an image.  It iterates a little to exclude the larger excursions (<i>i.e.</i>, bright stars, dropouts).  The results (a plane) can be used either directly to subtract a background from an image or, with fits to a set of difference images, to model a global set of background corrections to minimize differences.

In [2]:
rtn = mFitplane('M17/raw/2mass-atlas-990502s-j1430080.fits')

print(rtn)

{'status': '0', 'a': -0.00016525931270827687, 'b': 0.002731437131331532, 'c': 152.3555451225345, 'crpix1': 256.5, 'crpix2': 512.5, 'xmin': -251.5, 'xmax': 251.5, 'ymin': -512.5, 'ymax': 510.5, 'xcenter': -32.900088275277625, 'ycenter': -47.52564926466694, 'npixel': 282936, 'rms': 1.5194888436909284, 'boxx': 0.0, 'boxy': -1.0, 'boxwidth': 1023.0, 'boxheight': 503.0, 'boxang': 90.0}


There is no side effect; all of the output is in the above return structure.

In [13]:
import os
from IPython.display import HTML, display

try:
    os.mkdirs('work/M17')
except:
    pass

mBackground('M17/raw/2mass-atlas-990502s-j1430080.fits', 
            'work/M17/2mass-atlas-990502s-j1430080_fitplane.fits', 
            -0.00016525931270827687, 
            0.002731437131331532,  
            152.3555451225345,
            noAreas=True)

mViewer('-ct 1 -gray M17/raw/2mass-atlas-990502s-j1430080.fits \
         min max gaussian-log -out work/M17/2mass-atlas-990502s-j1430080.png',
        '', mode=2) 

rtn = mViewer('-ct 1 -gray work/M17/2mass-atlas-990502s-j1430080_fitplane.fits \
         min max gaussian-log -out work/M17/2mass-atlas-990502s-j1430080_fitplane.png',
        '', mode=2)

display(HTML("<table><tr><td><img src='work/M17/2mass-atlas-990502s-j1430080.png'></td> \
                         <td><img src='work/M17/2mass-atlas-990502s-j1430080_fitplane.png'></td></tr></table>"))

,


The difference is subtle, as is not surprising.  The image on the left is somewhat darker at the top than it is at the bottom whereas the one on the right is more uniform in its background from top to bottom.

&nbsp;<p/>

## mFitplane Error Handling

If mFitplane encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies an image that doesn't exist: 


In [12]:
rtn = mFitplane('M17/raw/unknown.fits')

print(rtn)

{'status': '1', 'msg': b'Image file M17/raw/unknown.fits missing or invalid FITS"]\n'}


&nbsp;

## Classic Montage: mFitplane as a Stand-Alone Program

### mFitplane Unix/Windows Command-line Arguments

<p>mFitplane can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mFitplane [-b border] [-d level] [-s statusfile] [-l(evel-only)] in.fits
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mFitplane can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mFitplane                                                            */
/*                                                                       */
/*  Montage is a set of general reprojection / coordinate-transform /    */
/*  mosaicking programs.  Any number of input images can be merged into  */
/*  an output FITS file.  The attributes of the input are read from the  */
/*  input files; the attributes of the output are read a combination of  */
/*  the command line and a FITS header template file.                    */
/*                                                                       */
/*  This module, mFitplane, is used in conjuction with mDiff and         */
/*  mBgModel to determine how overlapping images relate to each          */
/*  other.  It is assumed that difference images have matching structure */
/*  information and that what is left when you difference them is just   */
/*  the relative offsets, slopes, etc.  By fitting the difference image, */
/*  we obtain the 'correction' that needs to be applied to one or the    */
/*  other (or in part to both) to bring them together.                   */
/*                                                                       */
/*   char  *input_file     FITS file for plane fitting                   */
/*   int    levelOnly      Only fit for level difference not a full      */
/*                         plane with slopes                             */
/*   int    border         Exclude a pixel border from the fitting       */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mFitplaneReturn *mFitplane(char *input_file, int levelOnly, int border, int debug)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mFitplaneReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   double a;             // Plane fit coefficient for X axis.
   double b;             // Plane fit coefficient for Y axis.
   double c;             // Plane fit constant offset.
   double crpix1;        // X-axis pixel coordinate for center of region.
   double crpix2;        // Y-axis pixel coordinate for center of region.
   double xmin;          // Minimum X-axis value.
   double xmax;          // Maximum X-axis value.
   double ymin;          // Minimum Y-axis value.
   double ymax;          // Maximum Y-axis value.
   double xcenter;       // Center X location.
   double ycenter;       // Center Y location.
   int    npixel;        // Total number of pixels fit.
   double rms;           // RMS of fit (pixels with large offset values were excluded in fit).
   double boxx;          // Rectanguar bounding box X center.
   double boxy;          // Rectanguar bounding box Y center.
   double boxwidth;      // Rectanguar bounding box width.
   double boxheight;     // Rectanguar bounding box height.
   double boxang;        // Rectanguar bounding box rotation angle.
};
</pre>